In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from trajectory_visualization import plot_trajectory, sidebyside, shift_trajectory, overlay, shift_trajectory3D
from trajectory_visualization import plot_trajectories3D, plot_3Doverlay, plot_MSDorDeff, plot_MeanMSDorDeff, randtraj, multrandtraj
from trajectory_visualization import randtraj2, plot_Mean2DMSDsorDeff, plot_MSDorDeffLR, LRfor3D2D, fillin, prettify
from methods_of_Dcalc import Dpointder, Dlinalg, Dlinalgw

import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.optimize as opt
import scipy.stats as stat
from mpl_toolkits.mplot3d import Axes3D
from operator import itemgetter
import random
import numpy as np
import numpy.ma as ma
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D

pi = np.pi
sin = np.sin
cos = np.cos

In [2]:
# Type in datasets you would like to combine.  Adjust totvids to reflect the total number of videos. Also type in the
# umppx and fps in the conversion dictionary.

logplot = 'CON_40x_2D_130ms_logplot'
Mplot = 'CON_40x_2D_130ms_Mplot'
Dplot = 'CON_40x_2D_130ms_Dplot'
Hplot = 'CON_40x_2D_130ms_Hplot'
Hlogplot = 'CON_40x_2D_130ms_Hlogplot'

conversion = dict() #First element is the umppx, second is fps

conversion[1] = (0.30, 5.33, 1)
conversion[2] = (0.30, 5.37, 1)
conversion[3] = (0.30, 5.37, 1)
conversion[4] = (0.30, 5.31, 1)
conversion[5] = (0.30, 5.33, 1)
conversion[6] = (0.30, 5.33, 1)
conversion[7] = (0.30, 5.35, 1)

trajectory = dict()

trajectory[1] = np.genfromtxt('./CON_40x_2D_130ms_1_T.csv',
            delimiter =",")
# trajectory[2] = np.genfromtxt('./CON_40x_2D_130ms_2_T.csv',
#             delimiter =",")
# trajectory[3] = np.genfromtxt('./CON_40x_2D_130ms_3_T.csv',
#             delimiter =",")
# trajectory[4] = np.genfromtxt('./CON_40x_2D_130ms_4_T.csv',
#             delimiter =",")
# trajectory[5] = np.genfromtxt('./CON_40x_2D_130ms_5_T.csv',
#             delimiter =",")
# trajectory[6] = np.genfromtxt('./CON_40x_2D_130ms_6_T.csv',
#             delimiter =",")
# trajectory[7] = np.genfromtxt('./CON_40x_2D_130ms_7_T.csv',
#             delimiter =",")

totvids = 1

for num in range(1, totvids + 1):
    
    #Remove titles from columns
    #trajectory[num]=np.delete(trajectory[num], 0,0)
    #Remove the number column from dataset
    trajectory[num]=np.delete(trajectory[num],0,1)

In [3]:
trajectory[1]

array([[   1.   ,    0.   ,   54.333, ...,    3.393,    6.428,  142.317],
       [   1.   ,    1.   ,   52.425, ...,    3.971,    8.314,   57.37 ],
       [   2.   ,    0.   ,   86.485, ...,   11.053,   31.076,   21.372],
       ..., 
       [ 911.   ,  158.   ,   82.252, ...,    2.543,    4.411,  133.151],
       [ 912.   ,  157.   ,   52.384, ...,    2.854,    5.177,  105.6  ],
       [ 912.   ,  158.   ,   51.58 , ...,    2.373,    4.002,  126.118]])

In [4]:
rawframes = trajectory[1][:, 1]
#frames = np.linspace()
frames = np.linspace(min(rawframes),max(rawframes), max(rawframes)+1).astype(np.int64)

x = dict()
y = dict()
z = dict()
M1x = dict()
M1y = dict()
M1z = dict()
M2xy = dict()
M2xz = dict()
M2yz = dict()
M3 = dict()

#trajectory

In [5]:
dataset = dict()

particles = trajectory[1][:, 0]
total = int(max(particles))
total1 = total + 1
rawdataset = trajectory[1][:, :]
rawdataset[:, 2:4] = conversion[1][0] * rawdataset[:, 2:4]
rawdataset[:, 4] = conversion[1][2] * rawdataset[:, 4]

# Creates an array for each trajectory containing all xyz data
for num in range(1, total1):

    hold = np.where(particles == num)
    itindex = hold[0]
    min1 = min(itindex)
    max1 = max(itindex)
    dataset[num] = (rawdataset[min1:max1+1, 0:5])

In [6]:
#I need to first create code that will fill in the blanks in my arrays.  I need to decide how to fill them in as well.
def fillin2(data):

    shap = int(max(data[:, 1])) + 1
    shape1 = int(min(data[:, 1]))
    newshap = shap - shape1
    filledin = np.zeros((newshap, 5))
    filledin[0, :] = data[0, :]

    count = 0
    new = 0
    other = 0
    tot = 0

    for num in range(1, newshap):
        if data[num-new, 1]-data[num-new-1, 1] == 1:
            count = count + 1
            filledin[num, :] = data[num-new, :]
        elif data[num - new, 1]-data[num - new - 1, 1] > 1:
            new = new + 1
            iba = int(data[num - new+1, 1]-data[num - new, 1])
            togoin = data[num - new]
            togoin[1] = togoin[1] + 1
            filledin[num, :] = togoin
            # dataset[2] = np.insert(dataset[2], [num + new - 2], togoin, axis=0)

        else:
            other = other + 1
        tot = count + new + other

    return filledin

In [7]:
for num in range(1, total1):
    dataset[num] = fillin2(dataset[num])
    x[num] = np.zeros(frames.shape[0])
    x[num][int(dataset[num][0,1]):int(dataset[num][-1,1])+1] = dataset[num][:, 2]
    y[num] = np.zeros(frames.shape[0])
    y[num][int(dataset[num][0,1]):int(dataset[num][-1,1])+1] = dataset[num][:, 3]
    z[num] = np.zeros(frames.shape[0])
    z[num][int(dataset[num][0,1]):int(dataset[num][-1,1])+1] = dataset[num][:, 4]

In [8]:
for num in range(1, total1):
    dataset[num] = fillin2(dataset[num])
    M1x[num] = np.zeros(frames.shape[0])
    M1x[num][0:frames.shape[0]] = 0
    M1y[num] = np.zeros(frames.shape[0])
    M1y[num][0:frames.shape[0]] = 0
    M1z[num] = np.zeros(frames.shape[0])
    M1z[num][0:frames.shape[0]] = 0
    M2xy[num] = np.zeros(frames.shape[0])
    M2xy[num][0:frames.shape[0]] = 0
    M2xz[num] = np.zeros(frames.shape[0])
    M2xz[num][0:frames.shape[0]] = 0
    M2yz[num] = np.zeros(frames.shape[0])
    M2yz[num][0:frames.shape[0]] = 0
    M3[num] = np.zeros(frames.shape[0])
    M3[num][0:frames.shape[0]] = 0

In [9]:
I = dict()
for num in range(1, total1):

    I[num] = np.nonzero(x[num])[0]
    first = I[num][0]
    last = I[num][-1] + 1
    startx = x[num][first]
    starty = y[num][first]
    startz = z[num][first]


    for num1 in range(first, last):
        M1x[num][num1] = (x[num][num1] - startx)**2
        M1y[num][num1] = (y[num][num1] - starty)**2
        M1z[num][num1] = (z[num][num1] - startz)**2
        M2xy[num][num1] = (x[num][num1] - startx)**2 + (y[num][num1] - starty)**2
        M2xz[num][num1] = (x[num][num1] - startx)**2 + (z[num][num1] - startz)**2
        M2yz[num][num1] = (z[num][num1] - startz)**2 + (y[num][num1] - starty)**2
        M3[num][num1] = (z[num][num1] - startz)**2 + (y[num][num1] - starty)**2 + (x[num][num1] - startx)**2
        
#         M1x[num][first] = 'NaN'
#         M1y[num][first] = 'NaN'
#         M1z[num][first] = 'NaN'
#         M2xy[num][first] = 'NaN'
#         M2xz[num][first] = 'NaN'
#         M2yz[num][first] = 'NaN'
#         M3[num][first] = 'NaN'
    M1x[num] = ma.masked_equal(M1x[num], 0)
    M1y[num] = ma.masked_equal(M1y[num], 0)
    M1z[num] = ma.masked_equal(M1z[num], 0)
    M2xy[num] = ma.masked_equal(M2xy[num], 0)
    M2xz[num] = ma.masked_equal(M2xz[num], 0)
    M2yz[num] = ma.masked_equal(M2yz[num], 0)
    M3[num] = ma.masked_equal(M3[num], 0)

In [10]:
M1x[300]

masked_array(data = [-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 0.10949480999999749 1.1335860900000043 1.1335860900000043
 13.992836490000029 8.634782250000027 3.2996722500000177 -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --],
             mask = [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True False False False False False False  True  True  True  True  True
  True  True

In [11]:
stat.gmean([1, 2])

1.4142135623730949

In [12]:
#Each entry in yes is an array of particles that are in the given frame.  The frame is given by the index of yes.
yes = dict()
yessize = dict()
for num1 in range(1, frames.shape[0]):
    count = 0
    yes[num1] = np.zeros(frames.shape[0]).astype(np.int64)
    for num in range(1, total1):
        check = num1 in I[num]
        if check == True:
            yes[num1][count] = num
            count = count + 1
    iindex = np.where(yes[num1]==max(yes[num1]))[0][0]
    yes[num1] = yes[num1][0:iindex]
    

In [13]:
yes[158]

array([ 32, 226, 416, 572, 683, 758, 781, 787, 805, 825, 828, 839, 846,
       850, 856, 858, 861, 863, 866, 874, 875, 877, 880, 882, 883, 888,
       891, 892, 893, 897, 898, 899, 900, 901, 903, 904, 905, 906, 907,
       908, 909, 910, 911], dtype=int64)

In [ ]:
#M1x: first index is particle number, second index is frame
frame[0] = M1x[32][158]
frame[1] = M1x[226][158]

In [14]:
yes[158].shape[0]

43

In [34]:
#Here is my prototype for the frame-by-frame analysis.  I need to expand it in both directions.
frame = np.zeros(yes[158].shape[0])
for num1 in range(0, yes[158].shape[0]):
    frame[num1] = M1x[yes[158][num1]][155]
frame = ma.masked_invalid(frame)
stat.gmean(frame)

C:\Users\koolk\Miniconda3\lib\site-packages\numpy\ma\core.py:4085: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


0.21483045282018554

In [37]:
#First to make an array of datapoints based on the particles found in frame 158
frameM = np.zeros(frames.shape[0])
for num1 in range(0, frames.shape[0]):
    frame = np.zeros(yes[158].shape[0])
    for num in range(0, yes[158].shape[0]):
        frame[num] = M1x[yes[158][num]][num1]
    frame = ma.masked_invalid(frame)
    frameM[num1] = stat.gmean(frame)
frameM

C:\Users\koolk\Miniconda3\lib\site-packages\numpy\ma\core.py:4085: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


array([             nan,   1.01002500e-02,   6.22521000e-03,
         6.75684000e-03,   5.58009000e-03,   2.45025000e-03,
         5.40225000e-03,   5.18400000e-03,   6.20010000e-04,
         7.62129000e-03,   1.91844000e-03,   1.31769000e-03,
         6.27264000e-03,   3.56409000e-03,   1.71644490e-01,
         5.76081000e-03,   2.75625000e-03,   2.54016000e-03,
         4.31649000e-03,   2.78784000e-03,   3.18096000e-03,
         7.56900000e-03,   2.94849000e-03,   2.02500000e-03,
         3.85641000e-03,   2.48004000e-03,   4.59684000e-03,
         4.84416000e-03,   4.55625000e-03,   7.25904000e-03,
         1.90440000e-04,   1.58760000e-04,   8.29440000e-04,
         3.18096000e-03,   1.60821000e-03,   1.35864000e-03,
         2.25792000e-03,   2.70900000e-05,   1.40085000e-03,
         3.79890000e-03,   2.77920000e-04,   2.47050000e-04,
         3.45870000e-03,   1.92177000e-02,   2.93985000e-03,
         2.40570000e-04,   1.05300000e-03,   5.22900000e-03,
         4.34700000e-04,

In [53]:
#Now do do that same thing, but create an array for each frame
#First to make an array of datapoints based on the particles found in frame 158
framed = dict()
for num2 in range(1, frames.shape[0]):
    framed[num2] = np.zeros(frames.shape[0])
    for num1 in range(0, frames.shape[0]):
        frame = np.zeros(yes[num2].shape[0])
        for num in range(0, yes[num2].shape[0]):
            frame[num] = M1x[yes[num2][num]][num1]
        frame = ma.masked_invalid(frame)
        framed[num2][num1] = stat.gmean(frame)

C:\Users\koolk\Miniconda3\lib\site-packages\numpy\ma\core.py:4085: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


In [31]:
frames.shape[0]

159

In [54]:
framed[100]

array([             nan,   1.01002500e-02,   6.22521000e-03,
         6.75684000e-03,   5.58009000e-03,   2.45025000e-03,
         5.40225000e-03,   5.18400000e-03,   6.20010000e-04,
         7.62129000e-03,   1.91844000e-03,   1.31769000e-03,
         6.27264000e-03,   3.56409000e-03,   1.71644490e-01,
         5.76081000e-03,   2.75625000e-03,   2.54016000e-03,
         4.31649000e-03,   2.78784000e-03,   3.18096000e-03,
         7.56900000e-03,   2.94849000e-03,   2.02500000e-03,
         3.85641000e-03,   2.48004000e-03,   4.59684000e-03,
         4.84416000e-03,   4.55625000e-03,   7.25904000e-03,
         1.90440000e-04,   1.58760000e-04,   8.29440000e-04,
         3.18096000e-03,   1.60821000e-03,   1.35864000e-03,
         2.25792000e-03,   2.70900000e-05,   1.40085000e-03,
         3.79890000e-03,   2.77920000e-04,   2.47050000e-04,
         3.45870000e-03,   1.92177000e-02,   2.93985000e-03,
         2.40570000e-04,   1.05300000e-03,   5.22900000e-03,
         4.34700000e-04,

In [58]:
FBF = np.zeros(frames.shape[0])
for num in range(1, frames.shape[0]):
    FBF = FBF + framed[num]
FBF = FBF/frames.shape[0]
FBF

array([        nan,  0.02344454,  0.0540113 ,  0.07073489,  0.07522175,
        0.07331645,  0.09934608,  0.11890895,  0.07427568,  0.06078597,
        0.17434901,  0.24546088,  0.21120871,  0.33885929,  0.57341307,
        0.30526762,  0.23738198,  0.3452757 ,  0.40527176,  0.55168773,
        0.50239997,  0.5532829 ,  0.38871113,  0.14050326,  0.29724513,
        0.3299363 ,  0.40727068,  0.41940459,  0.52770232,  0.65135369,
        0.42997818,  0.57479683,  0.70653469,  0.86448717,  0.56868615,
        0.57828449,  0.71923266,  0.95746174,  0.63995627,  1.47569351,
        1.65010601,  1.80278606,  2.14923815,  1.3825686 ,  1.06879997,
        1.25184115,  1.1248684 ,  1.15685803,  0.89266553,  0.73652749,
        1.19974833,  0.87655649,  0.46588898,  0.72646006,  0.9805555 ,
        0.41665662,  0.56973023,  0.30228163,  0.39222177,  0.4430344 ,
        0.60605015,  0.47073451,  0.39804895,  0.49389472,  0.28481669,
        0.25459043,  0.46072468,  0.07952266,  0.17946819,  0.51

In [60]:
frames

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158], dtype=int64)